In [8]:
import json
import glob
# import numpy as np
# from PIL import Image, ImageSequence

oracle_trajectories_train = []
oracle_results = {
    "/plancraft/outputs/oracle_symb/train/0/*.json": [],
    "/plancraft/outputs/oracle_symb/val/0/*.json": [],
}
x, c = 0,0
for path in oracle_results.keys():
    for f in glob.glob(path):
        with open(f, "r") as file:
            traj = json.load(file)
        if (
            len(traj["model_trace"]["inventory_history"]) == len(traj["model_trace"]["action_history"])
        ):
            # ignore impossible actions
            oracle_results[path].append(traj)

In [22]:
from pydantic import BaseModel, field_validator

class SymbolicMoveAction(BaseModel):
    """ "Moves an item from one slot to another"""

    slot_from: int
    slot_to: int
    quantity: int

    action_type: str = "move"


    @field_validator("action_type", mode="before")
    def transform_quantity(cls, value) -> str:
        return "move"


SymbolicMoveAction(slot_from=1, slot_to=2, quantity=1, action_type="ahjajahja").model_dump()

{'slot_from': 1, 'slot_to': 2, 'quantity': 1, 'action_type': 'move'}

In [10]:
from plancraft.models.prompts import get_system_prompt
from collections import defaultdict

SYSTEM_PROMPT = get_system_prompt(actions=["move", "smelt"])

SYSTEM_PROMPT


'You are crafting in Minecraft. You need to decide on the next action.\n\nCrafting Grid: The crafting table is organized into a 3x3 grid. Each slot in the grid has a unique identifier:\n    - Top row: [A1] [A2] [A3]\n    - Middle row: [B1] [B2] [B3]\n    - Bottom row: [C1] [C2] [C3]\n\nThe output of the crafting process is placed in a designated output slot labeled [0] You cannot move or smelt items directly into slot [0]\n\nInventory Slots: The remaining inventory slots (outside of the crafting grid) are used for storing items. These slots are labeled as [I1] to [I36]\n\nActions:\n\t- move: Transfer a specific quantity of an item from one slot to another\n\t- smelt: Smelt an item in a furnace and moves the output to a specific slot\n\nFormat\n\t- `move: from [Source] to [Target] with quantity N`\n\t- `smelt: from [Source] to [Target] with quantity N`\n\nExample:\n    - `move: from [I2] to [A1] with quantity 3`\n    - `smelt: from [I5] to [I6] with quantity 1`\n\nConstraints:\n   - You

In [13]:
import os
from plancraft.environments.actions import convert_from_slot_index

from collections import defaultdict

PROMPT = get_system_prompt(actions = ["move", "smelt"])


def convert_obs_to_text(objective: str, inventory: list[dict]):
    return f"TASK: {objective}\ninventory={json.dumps(inventory)}"


def convert_action_to_text(action: dict):
    # {'action_type': 'move', 'slot_from': 17, 'slot_to': 1, 'quantity': 1}
    slot_from = convert_from_slot_index(action['slot_from'])
    slot_to = convert_from_slot_index(action['slot_to'])
    return f"{action['action_type']}: from {slot_from} to slot {slot_to} with quantity {action['quantity']}"


# convert action and inventory to dialogue history
def convert_trajectory_to_base_dialogue(traj: dict):
    dialogue = [{"role": "system", "content": SYSTEM_PROMPT}]
    objective = traj["model_trace"]["objective"]
    for action, inventory in zip(
        traj["model_trace"]["action_history"],
        traj["model_trace"]["inventory_history"],
    ):
        dialogue.append(
            {
                "role": "user",
                "content": convert_obs_to_text(objective, inventory),
            }
        )
        dialogue.append(
            {
                "role": "assistant",
                "content": convert_action_to_text(action),
            }
        )
    example = {
        "messages": dialogue,
        "example_id": traj["example_id"],
    }
    return example


convert_trajectory_to_base_dialogue(oracle_results["/plancraft/outputs/oracle_symb/train/0/*.json"][0])





KeyError: 'action_type'

In [ ]:
# convert action and inventory to dialogue history
def convert_trajectory_to_image_dialogue(traj: dict):
    dialogue = [
        {"role": "system", "content": [{"type": "text", "text": SYSTEM_PROMPT}]}
    ]
    objective = traj["model_trace"]["objective"]
    images = []
    for image, action, inventory in zip(
        traj["model_trace"]["images"],
        traj["model_trace"]["action_history"],
        traj["model_trace"]["inventory_history"],
    ):
        dialogue.append(
            {
                "role": "user",
                "content": [{"type": "image"}, {"type": "text", "text": objective}],
            }
        )
        dialogue.append(
            {
                "role": "assistant",
                "content": [{"type": "text", "text": convert_action_to_text(action)}],
            }
        )
        images.append(image)
    example = {
        "messages": dialogue,
        "example_id": traj["example_id"],
    }
    return example, images


In [ ]:
text_data = defaultdict(list)
mm_data = defaultdict(list)
for path, trajs in oracle_results.items():
    split = path.split("/")[-3]
    for traj in trajs:
        text_example = convert_trajectory_to_base_dialogue(traj)
        text_data[split].append(text_example)
        mm_example, example_imgs = convert_trajectory_to_image_dialogue(traj)
        mm_data[split].append(mm_example)
        # save imgs as png in format "data/oracle/{split}/{example_id}_{step}.gif"
        os.makedirs(f"data/oracle/{split}", exist_ok=True)
        for i, img in enumerate(example_imgs):
            Image.fromarray(img).save(
                f"data/oracle/{split}/{traj['example_id']}_{i}.png"
            )

    # save as jsonl file
    with open(f"data/oracle/{split}.jsonl", "w") as f:
        for example in text_data[split]:
            f.write(json.dumps(example) + "\n")

    with open(f"data/oracle/{split}.mm.jsonl", "w") as f:
        for example in mm_data[split]:
            f.write(json.dumps(example) + "\n")
